In [8]:
import os

import sklearn.linear_model

for dirname, _, filenames in os.walk("./data"):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./data/test.csv
./data/train.csv
./data/gender_submission.csv


In [18]:
import numpy as np
import pandas as pd

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

data = pd.concat([train, test], sort=False)
data["Sex"].replace(["male", "female"], [0, 1], inplace=True)
data["Embarked"].fillna("S", inplace=True)
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q":2}).astype(int)
data["Fare"].fillna(np.mean(data["Fare"]), inplace=True)

In [19]:
age_avg = data["Age"].mean()
age_std = data["Age"].std()
data["Age"].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)
drop_columns = ["Name", "PassengerId", "SibSp", "Parch", "Ticket", "Cabin"]
data.drop(drop_columns, axis=1, inplace=True)
train = data[:len(train)]
test = data[len(train):]
y_train = train["Survived"]
X_train = train.drop("Survived", axis=1)
X_test = test.drop("Survived", axis=1)

In [20]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2", solver="sag", random_state=0)
clf.fit(X_train, y_train)

/Users/tsumorishunsuke/PycharmProjects/my-kaggle/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(random_state=0, solver='sag')

In [21]:
y_pred = clf.predict(X_test)

In [22]:
y_pred[:20]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0.])

In [23]:
sub = pd.read_csv("data/gender_submission.csv")
sub["Survived"] = list(map(int, y_pred))
sub.to_csv("submission.csv", index=False)